In [1]:
#!  pip install nltk tensorflow tflearn
# nltk.download("punkt")
#!pip install --upgrade Pillow


In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
import tensorflow as tf
import numpy as np
import json
import random


In [2]:
# Initialize Lancaster Stemmer for word stemming
stemmer = LancasterStemmer()

# Load intents from intents.json
with open('intents.json') as file:
    data = json.load(file)

In [3]:
# Preprocess the data to extract words, labels, and create training data
words = []
labels = []
x_docs = []
y_docs = []

# Iterate through intents data to gather words, labels, and create training data
for intent in data['intents']:
    for pattern in intent['patterns']:
        # Tokenize each pattern into words
        wrds = nltk.word_tokenize(pattern)
        # Extend words list with the tokenized words
        words.extend(wrds)
        # Append tokenized words and intent tag to x_docs and y_docs respectively
        x_docs.append(wrds)
        y_docs.append(intent['tag'])
        # Add unique intent tags to labels list
        if intent['tag'] not in labels:
            labels.append(intent['tag'])


In [4]:
# Stemming words, converting to lowercase, and removing duplicate elements
words = [stemmer.stem(w.lower()) for w in words if w not in "?"]
words = sorted(list(set(words)))
labels = sorted(labels)

In [5]:
# Create training data using bag-of-words approach
training = []
output = []
out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(x_docs):
    bag = []
    wrds = [stemmer.stem(w) for w in doc]
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(y_docs[x])] = 1

    training.append(bag)
    output.append(output_row)

training = np.array(training)
output = np.array(output)

In [6]:
# Define and compile the neural network model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, input_shape=(len(training[0]),), activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(output[0]), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(training, output, epochs=100, batch_size=8)  # Adjust epochs and batch size as needed

# Save the trained model
model.save('chatbot_model.h5')

Epoch 1/100
4/4 [==============================] - 0s 0s/step - loss: 2.1929 - accuracy: 0.1290
Epoch 2/100
4/4 [==============================] - 0s 0s/step - loss: 2.0345 - accuracy: 0.1613
Epoch 3/100
4/4 [==============================] - 0s 0s/step - loss: 2.0058 - accuracy: 0.0968
Epoch 4/100
4/4 [==============================] - 0s 0s/step - loss: 1.9973 - accuracy: 0.1935
Epoch 5/100
4/4 [==============================] - 0s 0s/step - loss: 1.9653 - accuracy: 0.2903
Epoch 6/100
4/4 [==============================] - 0s 0s/step - loss: 2.0632 - accuracy: 0.1290
Epoch 7/100
4/4 [==============================] - 0s 0s/step - loss: 1.8570 - accuracy: 0.4194
Epoch 8/100
4/4 [==============================] - 0s 0s/step - loss: 1.9579 - accuracy: 0.2903
Epoch 9/100
4/4 [==============================] - 0s 5ms/step - loss: 1.8624 - accuracy: 0.2903
Epoch 10/100
4/4 [==============================] - 0s 0s/step - loss: 1.9540 - accuracy: 0.3226
Epoch 11/100
4/4 [====================

In [7]:
# Making predictions and creating chat function

def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for s_word in s_words:
        for i, w in enumerate(words):
            if w == s_word:
                bag[i] = 1

    return np.array(bag)

In [ ]:
def chat():
    while True:
        inp = input("\nYou: ")
        if inp.lower() == 'quit':
            break

        results = model.predict(np.array([bag_of_words(inp, words)]))
        results_index = np.argmax(results)
        tag = labels[results_index]

        for tg in data['intents']:
            if tg['tag'] == tag:
                responses = tg['responses']
                print("Bot:", random.choice(responses))

# Start the chat function
chat()


You:  hii


1/1 [==============================] - 0s 63ms/step
Bot: Good to see you again



You:  tell mi about you


1/1 [==============================] - 0s 7ms/step
Bot: I am here to assist you with information. My purpose is to help and provide guidance.



You:  what is todays whetar


1/1 [==============================] - 0s 16ms/step
Bot: I am here to assist you with information. My purpose is to help and provide guidance.
